In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\PIMA\data_2\cleaned_3.csv')


# Drop any unnecessary columns
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

df.head(2)

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,GenHlth_X_PhysHlth,BMI_X_PhysActivity,Age_X_HighBP,Income_X_Education,Age_X_DiffWalk,health_bp_decay,low_ses,condition_count,health_cluster,risk_score_mult
0,0.0,1.0,1.0,1.0,6,1.0,0.0,0.0,0.0,0.0,...,75.0,0.0,9.0,12.0,9.0,1.839397,0,3.0,0,14.715
1,0.0,0.0,0.0,0.0,3,1.0,0.0,0.0,1.0,0.0,...,0.0,3.0,0.0,6.0,0.0,3.000000,0,0.0,1,3.210


In [3]:
df['Diabetes_012'].value_counts()

Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

In [41]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Let's assume X and y are your features and target variable
# X = your_dataframe
# y = your_target_variable (with values 0, 1, 2)

# For demonstration, I'll generate some sample data
# In practice, you would use your actual data
y = df['Diabetes_012']
X = df.drop('Diabetes_012',axis=1)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

def calculate_class_weights(y):
    unique_classes = np.unique(y)
    class_counts = np.bincount(y.astype(int))
    total_samples = len(y)
    
    n_classes = len(unique_classes)
    weights = {}
    for i, cls in enumerate(unique_classes):
        weights[int(cls)] = total_samples / (n_classes * class_counts[int(cls)])
    return weights

# Get class weights
class_weights_dict = calculate_class_weights(y_train)

# Adjust weights based on your results
# Since class 2 already has good recall but class 1 is poor, increase weight for class 1
if 1 in class_weights_dict and 2 in class_weights_dict:
    class_weights_dict[1] = class_weights_dict[1] * 3  # Triple the weight for class 1
    # Don't modify class 2 weight since it's already performing well

print(f"Class weights: {class_weights_dict}")

# Initialize CatBoost classifier
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='MultiClass',
    class_weights=class_weights_dict,
    random_seed=42,
    verbose=100,
    eval_metric='AUC:type=Mu'
)

# Fit the model
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Get prediction probabilities
pred_probs = model.predict_proba(X_test)

# Improved thresholding function based on our findings
def refined_class_thresholds(probs):
    """
    Based on the results, we need to:
    1. Maintain good class 2 recall (was 90% without custom thresholds)
    2. Improve class 1 recall (was only 3.5%)
    3. Keep reasonable class 0 performance
    """
    # Let's try class-specific thresholds tuned to your dataset
    thresholds = [0.35, 0.15, 0.30]  # for classes 0, 1, 2
    
    # Initialize predictions array
    predictions = np.zeros(len(probs))
    
    # First priority: Identify class 1 (very poor recall in standard model)
    for i, prob_row in enumerate(probs):
        if prob_row[1] > thresholds[1]:
            predictions[i] = 1
        elif prob_row[2] > thresholds[2]:
            predictions[i] = 2
        elif prob_row[0] > thresholds[0]:
            predictions[i] = 0
        else:
            # For any remaining cases, use argmax
            predictions[i] = np.argmax(prob_row)
    
    return predictions

# Apply standard prediction
y_pred_standard = model.predict(X_test)

# Apply our refined thresholds
y_pred_refined = refined_class_thresholds(pred_probs)

# Also try a version focused purely on thresholds without changing order
def direct_class_thresholds(probs, thresholds=[0.4, 0.15, 0.3]):
    """Simple thresholding without prioritizing any class"""
    predictions = np.zeros(len(probs))
    
    for i, prob_row in enumerate(probs):
        # Check each class against its threshold
        for class_idx, threshold in enumerate(thresholds):
            if prob_row[class_idx] > threshold:
                # If probability exceeds class-specific threshold, assign this class
                predictions[i] = class_idx
                break
        
        # If no threshold was met, use argmax
        if predictions[i] == 0 and prob_row[0] <= thresholds[0]:
            predictions[i] = np.argmax(prob_row)
    
    return predictions

# Apply direct thresholds
y_pred_direct = direct_class_thresholds(pred_probs)

# Evaluate results
print("\nStandard Prediction Metrics:")
print(classification_report(y_test, y_pred_standard))

print("\nRefined Threshold Metrics (prioritizing class 1):")
print(classification_report(y_test, y_pred_refined))

print("\nDirect Threshold Metrics:")
print(classification_report(y_test, y_pred_direct))

# Calculate class-specific recalls
for strategy, y_pred in [("Standard", y_pred_standard), 
                         ("Refined", y_pred_refined), 
                         ("Direct", y_pred_direct)]:
    print(f"\n{strategy} prediction recalls:")
    for i in range(3):  # For classes 0, 1, 2
        if i in np.unique(y_test):  # Only calculate for classes that exist in test set
            recall = recall_score(y_test == i, y_pred == i)
            print(f"Class {i} Recall: {recall:.4f}")

# Try different threshold combinations to find optimal settings
print("\nExploring different threshold combinations:")
threshold_options = [
    [0.4, 0.1, 0.3],   # Lower threshold for class 1
    [0.4, 0.2, 0.3],   # Slightly higher threshold for class 1
    [0.35, 0.15, 0.35], # Higher threshold for class 2
    [0.4, 0.2, 0.25]    # Lower threshold for class 2
]

for thresholds in threshold_options:
    y_pred = direct_class_thresholds(pred_probs, thresholds)
    recalls = []
    for i in range(3):
        if i in np.unique(y_test):
            recall = recall_score(y_test == i, y_pred == i)
            recalls.append(recall)
    
    print(f"Thresholds {thresholds}: Class recalls: {[round(r, 4) for r in recalls]}")

Class weights: {0: 0.395690270352476, 1: 54.77570850202429, 2: 2.392332991477172}
0:	test: 0.6759476	best: 0.6759476 (0)	total: 142ms	remaining: 2m 21s
100:	test: 0.7348306	best: 0.7348306 (100)	total: 11.8s	remaining: 1m 45s
200:	test: 0.7379563	best: 0.7380058 (193)	total: 21.6s	remaining: 1m 26s
300:	test: 0.7377049	best: 0.7381395 (231)	total: 31.4s	remaining: 1m 12s
400:	test: 0.7365872	best: 0.7381395 (231)	total: 44.2s	remaining: 1m 6s
500:	test: 0.7352545	best: 0.7381395 (231)	total: 1m 5s	remaining: 1m 4s
600:	test: 0.7345064	best: 0.7381395 (231)	total: 1m 29s	remaining: 59.1s
700:	test: 0.7326145	best: 0.7381395 (231)	total: 1m 47s	remaining: 45.8s
800:	test: 0.7309489	best: 0.7381395 (231)	total: 1m 57s	remaining: 29.2s
900:	test: 0.7297242	best: 0.7381395 (231)	total: 2m 7s	remaining: 14s
999:	test: 0.7284599	best: 0.7381395 (231)	total: 2m 17s	remaining: 0us

bestTest = 0.7381395005
bestIteration = 231

Shrink model to first 232 iterations.

Standard Prediction Metrics:
 

In [44]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt



# Create binary target for first stage model (0 vs [1,2])
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test, y_binary_train, y_binary_test = train_test_split(
    X, y, y_binary, test_size=0.2, random_state=42, stratify=y
)

print(f"Training data shapes - X: {X_train.shape}, y: {y_train.shape}")
print(f"Class distribution in training set: {np.bincount(y_train.astype(int))}")
print(f"Binary class distribution in training set: {np.bincount(y_binary_train)}")

# ----- STAGE 1: Binary Classification Model (0 vs [1,2]) -----

# Calculate class weights for binary model
def calculate_binary_weights(y):
    counts = np.bincount(y)
    return {0: 1.0, 1: counts[0] / counts[1]}

binary_weights = calculate_binary_weights(y_binary_train)
print(f"Binary model class weights: {binary_weights}")

# Initialize first stage model (binary classifier)
binary_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',  # Binary classification
    class_weights=binary_weights,
    random_seed=42,
    verbose=100
)

# Train binary model
binary_model.fit(X_train, y_binary_train, eval_set=(X_test, y_binary_test))

# Get predictions from binary model
binary_probs = binary_model.predict_proba(X_test)[:, 1]  # Probability of being class 1 or 2

# Apply threshold to binary model (lower threshold to favor recall)
binary_threshold = 0.3  # This can be tuned
binary_preds = (binary_probs >= binary_threshold).astype(int)

# Evaluate binary model
print("\n----- Stage 1 Binary Model Evaluation -----")
print(classification_report(y_binary_test, binary_preds))
print(f"Recall for positive cases (1 or 2): {recall_score(y_binary_test, binary_preds):.4f}")

# ----- STAGE 2: Multiclass Model (1 vs 2) for positive cases only -----

# Filter training data to include only classes 1 and 2
mask_train = y_train > 0
X_train_positives = X_train[mask_train]
y_train_positives = y_train[mask_train]

# Adjust class values from [1,2] to [0,1] for the second model
y_train_positives = y_train_positives - 1

print(f"\nPositive cases for stage 2 training: {len(X_train_positives)}")
print(f"Class distribution in positive cases: {np.bincount(y_train_positives.astype(int))}")

# Calculate class weights for the second model
def calculate_positive_weights(y):
    counts = np.bincount(y)
    if len(counts) < 2:
        return {0: 1.0}  # Handle edge case
    return {0: 1.0, 1: counts[0] / counts[1]}

positive_weights = calculate_positive_weights(y_train_positives)
print(f"Positive cases model class weights: {positive_weights}")

# Initialize second stage model (classifier for positive cases)
positive_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',  # Binary between 1 and 2
    class_weights=positive_weights,
    random_seed=42,
    verbose=100
)

# Train positive cases model if we have enough samples
if len(np.unique(y_train_positives)) > 1 and len(y_train_positives) > 10:
    positive_model.fit(X_train_positives, y_train_positives)
    
    # Extract actual positive cases from test set for evaluation
    mask_test_actual_positives = y_test > 0
    X_test_actual_positives = X_test[mask_test_actual_positives]
    y_test_actual_positives = y_test[mask_test_actual_positives] - 1
    
    if len(X_test_actual_positives) > 0:
        # Get predictions on actual positive cases
        positive_probs = positive_model.predict_proba(X_test_actual_positives)[:, 1]
        
        # Apply threshold (lower threshold to favor class 2 recall)
        positive_threshold = 0.25  # This can be tuned
        positive_preds = (positive_probs >= positive_threshold).astype(int)
        
        # Evaluate positive model on actual positive cases
        print("\n----- Stage 2 Positive Cases Model Evaluation (on actual positives) -----")
        print(classification_report(y_test_actual_positives, positive_preds))
        
        # Class-specific recalls for the second model
        for cls in range(2):  # 0 means class 1, 1 means class 2
            recall = recall_score(y_test_actual_positives == cls, positive_preds == cls)
            print(f"Recall for class {cls+1}: {recall:.4f}")
else:
    print("\nWARNING: Not enough positive samples to train second stage model")

# ----- COMBINE MODELS FOR FINAL PREDICTION -----

# Function to make predictions using both models
def hierarchical_predict(X, binary_model, positive_model, binary_threshold=0.3, positive_threshold=0.25):
    # Stage 1: Predict if positive (class 1 or 2) or negative (class 0)
    binary_probs = binary_model.predict_proba(X)[:, 1]
    binary_preds = (binary_probs >= binary_threshold).astype(int)
    
    # Initialize with all zeros
    final_preds = np.zeros(len(X))
    
            # Find samples predicted as positive (1 or 2)
    positive_indices = np.where(binary_preds == 1)[0]
    
    if len(positive_indices) > 0:
        # Stage 2: For positive predictions, determine if class 1 or 2
        try:
            if hasattr(X, 'iloc'):
                X_positives = X.iloc[positive_indices]
            else:
                X_positives = X[positive_indices]
        except Exception as e:
            print(f"Error subsetting X positives: {e}")
            # Fallback approach
            if hasattr(X, 'loc'):
                X_positives = X.loc[positive_indices]
            else:
                X_positives = X[positive_indices, :]
        positive_probs = positive_model.predict_proba(X_positives)[:, 1]
        positive_preds = (positive_probs >= positive_threshold).astype(int)
        
        # Convert from [0,1] back to [1,2]
        positive_preds = positive_preds + 1
        
        # Assign to final predictions
        final_preds[positive_indices] = positive_preds
    
    return final_preds

# Make hierarchical predictions
y_pred_hierarchical = hierarchical_predict(
    X_test, binary_model, positive_model, 
    binary_threshold=binary_threshold, 
    positive_threshold=positive_threshold
)

# Evaluate final hierarchical model
print("\n----- Combined Hierarchical Model Evaluation -----")
print(classification_report(y_test, y_pred_hierarchical))

# Class-specific recall comparison
print("\nClass-specific Recall for Hierarchical Model:")
for cls in range(3):
    if cls in np.unique(y_test):
        recall = recall_score(y_test == cls, y_pred_hierarchical == cls)
        print(f"Class {cls} Recall: {recall:.4f}")

# ----- THRESHOLD TUNING -----

# Function to tune thresholds for both stages
def tune_hierarchical_thresholds(X, y, binary_model, positive_model):
    binary_probs = binary_model.predict_proba(X)[:, 1]
    
    # Mask for actual positive cases
    positive_mask = y > 0
    
    # Handle X depending on type (DataFrame or numpy array)
    try:
        if hasattr(X, 'iloc'):
            X_positive = X.iloc[positive_mask]
        else:
            X_positive = X[positive_mask]
    except Exception as e:
        print(f"Error subsetting X: {e}")
        print(f"X type: {type(X)}")
        print(f"Positive mask shape: {positive_mask.shape}")
        # Fallback approach
        if hasattr(X, 'loc'):
            X_positive = X.loc[positive_mask]
        else:
            indices = np.where(positive_mask)[0]
            X_positive = X[indices]
    
    y_positive = y[positive_mask] - 1  # Adjust to [0,1]
    
    if len(X_positive) > 0:
        positive_probs = positive_model.predict_proba(X_positive)[:, 1]
    else:
        return []
    
    results = []
    
    # Test different threshold combinations
    binary_thresholds = [0.2, 0.25, 0.3, 0.35, 0.4]
    positive_thresholds = [0.15, 0.2, 0.25, 0.3, 0.35]
    
    for b_thresh in binary_thresholds:
        for p_thresh in positive_thresholds:
            # Make predictions
            preds = hierarchical_predict(
                X, binary_model, positive_model,
                binary_threshold=b_thresh,
                positive_threshold=p_thresh
            )
            
            # Calculate class-specific recalls
            recalls = []
            for cls in range(3):
                if cls in np.unique(y):
                    recall = recall_score(y == cls, preds == cls)
                    recalls.append(recall)
                else:
                    recalls.append(np.nan)
            
            # Store results
            results.append({
                'binary_threshold': b_thresh,
                'positive_threshold': p_thresh,
                'class0_recall': recalls[0],
                'class1_recall': recalls[1] if len(recalls) > 1 else np.nan,
                'class2_recall': recalls[2] if len(recalls) > 2 else np.nan,
            })
    
    return results

# Tune thresholds
threshold_results = tune_hierarchical_thresholds(X_test, y_test, binary_model, positive_model)

# Sort results by different criteria
print("\n----- Threshold Tuning Results -----")
print("Top 5 settings for class 2 recall:")
top_class2 = sorted(threshold_results, key=lambda x: x['class2_recall'], reverse=True)[:5]
for result in top_class2:
    print(f"Binary threshold: {result['binary_threshold']}, Positive threshold: {result['positive_threshold']}")
    print(f"  Class 0 recall: {result['class0_recall']:.4f}")
    print(f"  Class 1 recall: {result['class1_recall']:.4f}")
    print(f"  Class 2 recall: {result['class2_recall']:.4f}")
    print()

print("Top 5 settings for balanced recall (prioritizing classes 1 and 2):")
def balance_score(result):
    # Weighted average with emphasis on classes 1 and 2
    return result['class1_recall'] * 0.4 + result['class2_recall'] * 0.5 + result['class0_recall'] * 0.1

top_balanced = sorted(threshold_results, key=balance_score, reverse=True)[:5]
for result in top_balanced:
    print(f"Binary threshold: {result['binary_threshold']}, Positive threshold: {result['positive_threshold']}")
    print(f"  Class 0 recall: {result['class0_recall']:.4f}")
    print(f"  Class 1 recall: {result['class1_recall']:.4f}")
    print(f"  Class 2 recall: {result['class2_recall']:.4f}")
    print()

# Save models
binary_model.save_model('catboost_binary_diabetes.cbm')
positive_model.save_model('catboost_positive_diabetes.cbm')

# Create a function to apply the hierarchical model to new data
def predict_diabetes_severity(X_new, binary_model_path='catboost_binary_diabetes.cbm', 
                             positive_model_path='catboost_positive_diabetes.cbm',
                             binary_threshold=0.3, positive_threshold=0.25):
    """
    Predict diabetes severity for new data using the hierarchical model.
    
    Parameters:
    -----------
    X_new : DataFrame or array
        New features to predict on
    binary_model_path : str
        Path to saved binary model
    positive_model_path : str
        Path to saved positive cases model
    binary_threshold : float
        Threshold for binary model
    positive_threshold : float
        Threshold for positive cases model
    
    Returns:
    --------
    numpy.ndarray
        Predictions: 0 = no diabetes, 1 = moderate diabetes, 2 = severe diabetes
    """
    # Load models
    binary_model = CatBoostClassifier()
    binary_model.load_model(binary_model_path)
    
    positive_model = CatBoostClassifier()
    positive_model.load_model(positive_model_path)
    
    # Make predictions
    return hierarchical_predict(X_new, binary_model, positive_model, 
                               binary_threshold, positive_threshold)

print("\nExample usage of prediction function:")
print("predictions = predict_diabetes_severity(new_data)")

Training data shapes - X: (202944, 33), y: (202944,)
Class distribution in training set: [170962   3705  28277]
Binary class distribution in training set: [170962  31982]
Binary model class weights: {0: 1.0, 1: 5.345569382777812}
0:	learn: 0.6762877	test: 0.6766480	best: 0.6766480 (0)	total: 133ms	remaining: 2m 13s
100:	learn: 0.5095463	test: 0.5151712	best: 0.5151712 (100)	total: 6.07s	remaining: 54.1s
200:	learn: 0.5051043	test: 0.5129590	best: 0.5129401 (198)	total: 11.5s	remaining: 45.7s
300:	learn: 0.5014517	test: 0.5122024	best: 0.5122024 (300)	total: 15.9s	remaining: 37s
400:	learn: 0.4979810	test: 0.5119325	best: 0.5119278 (392)	total: 19.9s	remaining: 29.8s
500:	learn: 0.4949839	test: 0.5119471	best: 0.5119278 (392)	total: 23.8s	remaining: 23.7s
600:	learn: 0.4922731	test: 0.5123480	best: 0.5119278 (392)	total: 27.8s	remaining: 18.4s
700:	learn: 0.4897417	test: 0.5127385	best: 0.5119278 (392)	total: 31.7s	remaining: 13.5s
800:	learn: 0.4873490	test: 0.5130814	best: 0.5119278 (

In [46]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# Load data
data = pd.read_csv(r'D:\PIMA\data_2\cleaned_3.csv')  # Adjust path as needed

# Set up features and target
X = data.drop('Diabetes_012', axis=1)  # Adjust column name as needed
y = data['Diabetes_012'].astype(int)  # Ensure this has 0, 1, 2 values

# Create binary target for first stage model (0 vs [1,2])
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test, y_binary_train, y_binary_test = train_test_split(
    X, y, y_binary, test_size=0.2, random_state=42, stratify=y
)

print(f"Class distribution in training set: {np.bincount(y_train.astype(int))}")
print(f"Binary class distribution in training set: {np.bincount(y_binary_train)}")

# ----- STAGE 1: Binary Classification Model (0 vs [1,2]) -----

# Calculate class weights for binary model
binary_weights = {0: 1.0, 1: 5.0}  # Substantial weight to positives
print(f"Binary model class weights: {binary_weights}")

# Initialize first stage model (binary classifier)
binary_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    class_weights=binary_weights,
    random_seed=42,
    verbose=100
)

# Train binary model
binary_model.fit(X_train, y_binary_train, eval_set=(X_test, y_binary_test))

# Get predictions from binary model
binary_probs = binary_model.predict_proba(X_test)[:, 1]

# Apply threshold to binary model (lower threshold to favor recall)
binary_threshold = 0.2  # Even lower threshold to catch more positive cases
binary_preds = (binary_probs >= binary_threshold).astype(int)

# Evaluate binary model
print("\n----- Stage 1 Binary Model Evaluation -----")
print(classification_report(y_binary_test, binary_preds))
print(f"Recall for positive cases (1 or 2): {recall_score(y_binary_test, binary_preds):.4f}")

# ----- STAGE 2: Multiclass Model (1 vs 2) for positive cases only -----

# Filter training data to include only classes 1 and 2
mask_train = y_train > 0
X_train_positives = X_train[mask_train]
y_train_positives = y_train[mask_train]

# Adjust class values from [1,2] to [0,1] for the second model
y_train_positives = y_train_positives - 1

print(f"\nClass distribution in positive cases: {np.bincount(y_train_positives.astype(int))}")

# Apply SMOTE to balance classes 1 and 2
try:
    print("Applying SMOTE to balance positive classes...")
    smote = SMOTE(random_state=42)
    X_train_pos_resampled, y_train_pos_resampled = smote.fit_resample(X_train_positives, y_train_positives)
    print(f"After SMOTE - Class distribution: {np.bincount(y_train_pos_resampled.astype(int))}")
except Exception as e:
    print(f"Error applying SMOTE: {e}")
    print("Falling back to class weights...")
    X_train_pos_resampled, y_train_pos_resampled = X_train_positives, y_train_positives

# Use extreme class weights to favor class 1 (minority class)
positive_weights = {0: 8.0, 1: 1.0}  # Much higher weight for class 1 (which is class 0 after adjustment)
print(f"Positive cases model class weights: {positive_weights}")

# Initialize second stage model with strong focus on class 1
positive_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,  # Lower learning rate for better generalization
    depth=6,
    loss_function='Logloss',
    class_weights=positive_weights,
    random_seed=42,
    verbose=100
)

# Train positive cases model on resampled data
positive_model.fit(X_train_pos_resampled, y_train_pos_resampled)

# Extract actual positive cases from test set for evaluation
mask_test_actual_positives = y_test > 0
X_test_actual_positives = X_test[mask_test_actual_positives]
y_test_actual_positives = y_test[mask_test_actual_positives] - 1

# Get predictions on actual positive cases
positive_probs = positive_model.predict_proba(X_test_actual_positives)

# Try 5 different thresholds focused on class 1 recall
thresholds = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
best_f1 = 0
best_threshold = 0.5
best_recalls = (0, 0)

print("\n----- Testing thresholds for Stage 2 model -----")
for threshold in thresholds:
    # For class 1 (which is 0 after adjustment), if prob < threshold, predict class 1
    # This means we're making it easier to predict class 1
    positive_preds = (positive_probs[:, 0] >= threshold).astype(int)  # Lower threshold for class 1
    positive_preds = 1 - positive_preds  # Invert to make 0 = class 1 (original class 1) and 1 = class 2 (original class 2)
    
    recall_0 = recall_score(y_test_actual_positives == 0, positive_preds == 0)
    recall_1 = recall_score(y_test_actual_positives == 1, positive_preds == 1)
    f1 = (recall_0 * 0.7 + recall_1 * 0.3)  # Weighted toward class 1 recall
    
    print(f"Threshold {threshold:.2f}: Class 1 recall: {recall_0:.4f}, Class 2 recall: {recall_1:.4f}, Combined: {f1:.4f}")
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_recalls = (recall_0, recall_1)

print(f"\nBest threshold: {best_threshold:.2f} with recalls - Class 1: {best_recalls[0]:.4f}, Class 2: {best_recalls[1]:.4f}")

# Apply best threshold for final model
positive_threshold = best_threshold

# Function to make hierarchical predictions with the optimized thresholds
def hierarchical_predict(X, binary_model, positive_model, binary_threshold=0.2, positive_threshold=0.3):
    # Stage 1: Predict if diabetes (class 1 or 2) or not (class 0)
    binary_probs = binary_model.predict_proba(X)[:, 1]
    binary_preds = (binary_probs >= binary_threshold).astype(int)
    
    # Initialize with all zeros
    final_preds = np.zeros(len(X))
    
    # Find samples predicted as positive (1 or 2)
    positive_indices = np.where(binary_preds == 1)[0]
    
    if len(positive_indices) > 0:
        # Stage 2: For positive predictions, determine if class 1 or 2
        X_positives = X.iloc[positive_indices] if hasattr(X, 'iloc') else X[positive_indices]
        
        # Get probabilities for the adjusted classes (0=class1, 1=class2)
        positive_probs = positive_model.predict_proba(X_positives)
        
        # Apply inverted threshold: if prob of class1 (adjusted 0) >= threshold, predict class 1, else class 2
        # This prioritizes class 1 prediction
        positive_preds = (positive_probs[:, 0] >= positive_threshold).astype(int)
        
        # Convert back from [0,1] to [1,2] with the meaning that 0 means class 1, 1 means class 2
        positive_preds = positive_preds * 0 + (1 - positive_preds) * 1 + 1
        
        # Assign to final predictions
        final_preds[positive_indices] = positive_preds
    
    return final_preds

# Make hierarchical predictions with optimized thresholds
y_pred_hierarchical = hierarchical_predict(
    X_test, binary_model, positive_model, 
    binary_threshold=binary_threshold, 
    positive_threshold=positive_threshold
)

# Evaluate final hierarchical model
print("\n----- Combined Hierarchical Model Evaluation -----")
print(classification_report(y_test, y_pred_hierarchical))

# Class-specific recall analysis
print("\nClass-specific Recall for Hierarchical Model:")
for cls in range(3):
    if cls in np.unique(y_test):
        recall = recall_score(y_test == cls, y_pred_hierarchical == cls)
        print(f"Class {cls} Recall: {recall:.4f}")

# Save the optimized models
binary_model.save_model('catboost_binary_diabetes_optimized.cbm')
positive_model.save_model('catboost_positive_diabetes_optimized.cbm')

# Create a function to apply the optimized hierarchical model to new data
def predict_diabetes_severity(X_new, binary_model_path='catboost_binary_diabetes_optimized.cbm', 
                             positive_model_path='catboost_positive_diabetes_optimized.cbm',
                             binary_threshold=0.2, positive_threshold=best_threshold):
    """
    Predict diabetes severity for new data using the optimized hierarchical model.
    
    Parameters:
    -----------
    X_new : DataFrame or array
        New features to predict on
    binary_model_path : str
        Path to saved binary model
    positive_model_path : str
        Path to saved positive cases model
    binary_threshold : float
        Threshold for binary model
    positive_threshold : float
        Threshold for positive cases model
    
    Returns:
    --------
    numpy.ndarray
        Predictions: 0 = no diabetes, 1 = moderate diabetes, 2 = severe diabetes
    """
    # Load models
    binary_model = CatBoostClassifier()
    binary_model.load_model(binary_model_path)
    
    positive_model = CatBoostClassifier()
    positive_model.load_model(positive_model_path)
    
    # Make predictions with optimized thresholds
    return hierarchical_predict(X_new, binary_model, positive_model, 
                               binary_threshold, positive_threshold)

Class distribution in training set: [170962   3705  28277]
Binary class distribution in training set: [170962  31982]
Binary model class weights: {0: 1.0, 1: 5.0}
0:	learn: 0.6757686	test: 0.6759770	best: 0.6759770 (0)	total: 46ms	remaining: 46s
100:	learn: 0.5097676	test: 0.5155124	best: 0.5155124 (100)	total: 6.3s	remaining: 56.1s
200:	learn: 0.5052307	test: 0.5133545	best: 0.5133545 (200)	total: 11.4s	remaining: 45.2s
300:	learn: 0.5013412	test: 0.5124749	best: 0.5124749 (300)	total: 16.1s	remaining: 37.5s
400:	learn: 0.4977935	test: 0.5122168	best: 0.5121960 (399)	total: 21s	remaining: 31.4s
500:	learn: 0.4945215	test: 0.5122243	best: 0.5120909 (463)	total: 25.5s	remaining: 25.4s
600:	learn: 0.4915330	test: 0.5123922	best: 0.5120909 (463)	total: 30.6s	remaining: 20.3s
700:	learn: 0.4888457	test: 0.5125133	best: 0.5120909 (463)	total: 35.4s	remaining: 15.1s
800:	learn: 0.4861168	test: 0.5127654	best: 0.5120909 (463)	total: 41.6s	remaining: 10.3s
900:	learn: 0.4834734	test: 0.5131256

In [48]:
df['Diabetes_012'].value_counts()

Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

In [51]:
from sklearn.naive_bayes import GaussianNB

df = pd.read_csv(r'D:\PIMA\data_2\cleaned_3.csv')

# Drop any unnecessary columns
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

y = df['Diabetes_012']
X = df.drop('Diabetes_012',axis=1)



# Get the actual class distribution
class_counts = np.bincount(y.astype(int))
print(f"Class distribution in full dataset:")
for cls, count in enumerate(class_counts):
    print(f"Class {cls}: {count} samples ({count/len(y)*100:.2f}%)")

# Create binary target for first stage model (0 vs [1,2])
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test, y_binary_train, y_binary_test = train_test_split(
    X, y, y_binary, test_size=0.2, random_state=42, stratify=y
)

print(f"Class distribution in training set: {np.bincount(y_train.astype(int))}")
print(f"Binary class distribution in training set: {np.bincount(y_binary_train)}")

# ----- STAGE 1: Binary Classification Model (0 vs [1,2]) -----

# Calculate class weights for binary model - use weights that worked well previously
binary_weights = {0: 1.0, 1: 3.0}  # Adjust if needed based on performance
print(f"Binary model class weights: {binary_weights}")

# Initialize first stage model (binary classifier)
binary_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    loss_function='Logloss',
    class_weights=binary_weights,
    random_seed=42,
    verbose=100
)

# Train binary model
binary_model.fit(X_train, y_binary_train, eval_set=(X_test, y_binary_test))

# Get predictions from binary model
binary_probs = binary_model.predict_proba(X_test)[:, 1]

# Apply threshold to binary model
binary_threshold = 0.2  # Adjust based on your preferred recall/precision balance
binary_preds = (binary_probs >= binary_threshold).astype(int)

# Evaluate binary model
print("\n----- Stage 1 Binary Model Evaluation -----")
print(classification_report(y_binary_test, binary_preds))
print(f"Recall for positive cases (1 or 2): {recall_score(y_binary_test, binary_preds):.4f}")

# ----- STAGE 2: Naive Bayes for Severity Classification (1 vs 2) -----

# Filter training data to include only classes 1 and 2
mask_train = y_train > 0
X_train_positives = X_train[mask_train]
y_train_positives = y_train[mask_train] - 1  # Adjust to 0 (was 1) and 1 (was 2)

print(f"\nClass distribution in positive cases (training): {np.bincount(y_train_positives.astype(int))}")

# Initialize Naive Bayes model for severity classification
nb_model = GaussianNB()

# We can adjust the prior probabilities to account for class imbalance
# Calculate class distribution in positive cases
pos_class_counts = np.bincount(y_train_positives.astype(int))
pos_class_priors = pos_class_counts / pos_class_counts.sum()

# You can adjust priors to give more weight to the minority class (class 1)
# For example, you might want a 60/40 or 70/30 prior instead of the actual distribution
adjusted_priors = np.array([0.8, 0.2])  # Adjusted to give more weight to class 1 (40% vs actual ~10%)
print(f"Natural class priors: {pos_class_priors}")
print(f"Adjusted class priors: {adjusted_priors}")

# Set the adjusted priors (comment this out to use natural priors)
nb_model.priors = adjusted_priors

# Train Naive Bayes model
nb_model.fit(X_train_positives, y_train_positives)

# Extract actual positive cases from test set for evaluation
mask_test_actual_positives = y_test > 0
X_test_actual_positives = X_test[mask_test_actual_positives]
y_test_actual_positives = y_test[mask_test_actual_positives] - 1  # Adjust to 0 and 1

# Get predictions on actual positive cases
nb_probs = nb_model.predict_proba(X_test_actual_positives)
nb_preds = nb_model.predict(X_test_actual_positives)

# Function to explore different probability thresholds
def try_probability_thresholds(probs, y_true):
    results = []
    thresholds = np.linspace(0.1, 0.9, 17)  # Test 17 thresholds
    
    for threshold in thresholds:
        # Predict class 1 (severe) if probability > threshold
        preds = (probs[:, 1] >= threshold).astype(int)
        
        # Calculate metrics
        class0_recall = recall_score(y_true == 0, preds == 0)
        class1_recall = recall_score(y_true == 1, preds == 1)
        
        # Balanced score (equal weight to both classes)
        balanced_score = (class0_recall + class1_recall) / 2
        
        results.append({
            'threshold': threshold,
            'class1_recall': class0_recall,  # This is original class 1 (moderate)
            'class2_recall': class1_recall,  # This is original class 2 (severe)
            'balanced_score': balanced_score
        })
    
    return results

# Try different thresholds
threshold_results = try_probability_thresholds(nb_probs, y_test_actual_positives)

# Print top 5 thresholds by balanced score
print("\n----- Naive Bayes Threshold Exploration -----")
for result in sorted(threshold_results, key=lambda x: x['balanced_score'], reverse=True)[:5]:
    print(f"Threshold {result['threshold']:.2f}: "
          f"Class 1 recall: {result['class1_recall']:.4f}, "
          f"Class 2 recall: {result['class2_recall']:.4f}, "
          f"Balanced score: {result['balanced_score']:.4f}")

# Find best threshold
best_result = max(threshold_results, key=lambda x: x['balanced_score'])
nb_threshold = best_result['threshold']
print(f"\nBest threshold: {nb_threshold:.2f}")

# Apply threshold to get final predictions
nb_preds_threshold = (nb_probs[:, 1] >= nb_threshold).astype(int)

# Evaluate Naive Bayes model with best threshold
print("\n----- Stage 2 Naive Bayes Evaluation (on actual positives) -----")
print("Standard predictions:")
print(classification_report(y_test_actual_positives, nb_preds))
print("\nThreshold-optimized predictions:")
print(classification_report(y_test_actual_positives, nb_preds_threshold))

# Function to make hierarchical predictions
def hierarchical_predict(X, binary_model, nb_model, binary_threshold=0.3, nb_threshold=0.5):
    # Stage 1: Binary classification
    binary_probs = binary_model.predict_proba(X)[:, 1]
    binary_preds = (binary_probs >= binary_threshold).astype(int)
    
    # Initialize with all zeros
    final_preds = np.zeros(len(X))
    
    # Find samples predicted as positive (1 or 2)
    positive_indices = np.where(binary_preds == 1)[0]
    
    if len(positive_indices) > 0:
        # Stage 2: For positive predictions, determine severity
        if hasattr(X, 'iloc'):
            X_positives = X.iloc[positive_indices]
        else:
            X_positives = X[positive_indices]
        
        # Get probabilities from Naive Bayes
        nb_probs = nb_model.predict_proba(X_positives)
        
        # Apply threshold
        nb_preds = (nb_probs[:, 1] >= nb_threshold).astype(int)
        
        # Convert back from [0,1] to [1,2]
        severity_preds = nb_preds + 1
        
        # Assign to final predictions
        final_preds[positive_indices] = severity_preds
    
    return final_preds

# Make hierarchical predictions
y_pred_hierarchical = hierarchical_predict(
    X_test, binary_model, nb_model, 
    binary_threshold=binary_threshold, 
    nb_threshold=nb_threshold
)

# Evaluate final hierarchical model
print("\n----- Combined Hierarchical Model Evaluation -----")
print(classification_report(y_test, y_pred_hierarchical))

# Class-specific recall analysis
print("\nClass-specific Recall for Hierarchical Model:")
for cls in range(3):
    if cls in np.unique(y_test):
        recall = recall_score(y_test == cls, y_pred_hierarchical == cls)
        print(f"Class {cls} Recall: {recall:.4f}")

# Feature importance analysis for Naive Bayes
def get_naive_bayes_feature_importance(nb_model, feature_names):
    # For Gaussian Naive Bayes, we can look at the difference in means between classes
    # as an indication of feature importance
    mean_diff = np.abs(nb_model.theta_[1] - nb_model.theta_[0])
    
    # Create a DataFrame with feature names and importance scores
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': mean_diff
    })
    
    # Sort by importance
    return importance_df.sort_values('Importance', ascending=False)

# Get feature importance for Naive Bayes
nb_importance = get_naive_bayes_feature_importance(nb_model, X.columns)
print("\n----- Top 10 Important Features for Severity Classification -----")
print(nb_importance.head(10))

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = nb_importance.head(15)
plt.barh(top_features['Feature'][::-1], top_features['Importance'][::-1])
plt.xlabel('Importance (Mean Difference Between Classes)')
plt.title('Top 15 Features for Distinguishing Diabetes Severity')
plt.tight_layout()
plt.savefig('naive_bayes_feature_importance.png')
plt.close()

# Save the models
binary_model.save_model('catboost_binary_diabetes.cbm')
import joblib
joblib.dump(nb_model, 'naive_bayes_severity.joblib')
print("\nModels saved.")

# Create a function for making predictions on new data
def predict_diabetes_severity(X_new, binary_model_path='catboost_binary_diabetes.cbm', 
                             nb_model_path='naive_bayes_severity.joblib',
                             binary_threshold=0.3, nb_threshold=nb_threshold):
    """
    Predict diabetes severity for new data using the hierarchical model.
    
    Returns:
    --------
    numpy.ndarray
        Predictions: 0 = no diabetes, 1 = moderate diabetes, 2 = severe diabetes
    """
    # Load models
    binary_model = CatBoostClassifier()
    binary_model.load_model(binary_model_path)
    
    nb_model = joblib.load(nb_model_path)
    
    # Make predictions
    return hierarchical_predict(X_new, binary_model, nb_model, 
                               binary_threshold, nb_threshold)

print("\nExample usage of prediction function:")
print("predictions = predict_diabetes_severity(new_data)")

# Let's also check how well we're doing on precision for each class
def get_precision_by_threshold(probs, y_true):
    from sklearn.metrics import precision_score
    
    results = []
    thresholds = np.linspace(0.1, 0.9, 17)
    
    for threshold in thresholds:
        preds = (probs[:, 1] >= threshold).astype(int)
        
        # Calculate precision when available (handle division by zero)
        try:
            class0_precision = precision_score(y_true == 0, preds == 0)
        except:
            class0_precision = 0
            
        try:
            class1_precision = precision_score(y_true == 1, preds == 1)
        except:
            class1_precision = 0
        
        # F1-like score that balances recall and precision
        results.append({
            'threshold': threshold,
            'class1_precision': class0_precision,
            'class2_precision': class1_precision
        })
    
    return results

# Check precision at different thresholds
precision_results = get_precision_by_threshold(nb_probs, y_test_actual_positives)

print("\n----- Precision at Different Thresholds -----")
for result in precision_results:
    print(f"Threshold {result['threshold']:.2f}: "
          f"Class 1 precision: {result['class1_precision']:.4f}, "
          f"Class 2 precision: {result['class2_precision']:.4f}")

Class distribution in full dataset:
Class 0: 213703 samples (84.24%)
Class 1: 4631 samples (1.83%)
Class 2: 35346 samples (13.93%)
Class distribution in training set: [170962   3705  28277]
Binary class distribution in training set: [170962  31982]
Binary model class weights: {0: 1.0, 1: 3.0}
0:	learn: 0.6799525	test: 0.6801143	best: 0.6801143 (0)	total: 45.8ms	remaining: 45.8s
100:	learn: 0.4903567	test: 0.4944139	best: 0.4944139 (100)	total: 4.77s	remaining: 42.4s
200:	learn: 0.4851599	test: 0.4905256	best: 0.4905256 (200)	total: 9.06s	remaining: 36s
300:	learn: 0.4828744	test: 0.4894473	best: 0.4894473 (300)	total: 13.7s	remaining: 31.8s
400:	learn: 0.4809362	test: 0.4889510	best: 0.4889507 (399)	total: 20.2s	remaining: 30.1s
500:	learn: 0.4788907	test: 0.4886254	best: 0.4886208 (499)	total: 25.8s	remaining: 25.7s
600:	learn: 0.4769740	test: 0.4885239	best: 0.4885195 (593)	total: 31.5s	remaining: 20.9s
700:	learn: 0.4752559	test: 0.4884828	best: 0.4884754 (698)	total: 36.4s	remainin

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Display column names for reference
print("Available features:", X.columns.tolist())

# Create binary target for first stage model (0 vs [1,2])
y_binary = (y > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test, y_binary_train, y_binary_test = train_test_split(
    X, y, y_binary, test_size=0.2, random_state=42, stratify=y
)

print(f"Class distribution in training set: {np.bincount(y_train.astype(int))}")
print(f"Binary class distribution in training set: {np.bincount(y_binary_train)}")

# ----- STAGE 1: Binary Classification Model (0 vs [1,2]) -----

# Initialize first stage model (binary classifier)
binary_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    class_weights={0: 1.0, 1: 3.0},
    random_seed=42,
    verbose=100
)

# Train binary model
binary_model.fit(X_train, y_binary_train, eval_set=(X_test, y_binary_test))

# Get predictions from binary model
binary_probs = binary_model.predict_proba(X_test)[:, 1]
binary_threshold = 0.2 # Use the threshold that gave good recall
binary_preds = (binary_probs >= binary_threshold).astype(int)

# Evaluate binary model
print("\n----- Stage 1 Binary Model Evaluation -----")
print(classification_report(y_binary_test, binary_preds))
print(f"Recall for positive cases (1 or 2): {recall_score(y_binary_test, binary_preds):.4f}")

# ----- STAGE 2: Deterministic Rules for Severity Classification -----

# Define deterministic rules function
def determine_diabetes_severity(X):
    """
    Apply clinical rules to determine diabetes severity (class 1 or 2)
    
    Rules are based on comorbidities and risk factors from the BRFSS dataset
    Returns 1 for moderate diabetes, 2 for severe diabetes
    """
    # Initialize severity scores
    severity_scores = np.zeros(len(X))
    
    # Rule 1: Heart disease or stroke indicates severe diabetes
    if 'HeartDiseaseorAttack' in X.columns:
        severity_scores += (X['HeartDiseaseorAttack'] == 1).astype(int) * 2
    
    if 'Stroke' in X.columns:
        severity_scores += (X['Stroke'] == 1).astype(int) * 2
    
    # Rule 2: BMI > 35 indicates higher severity
    if 'BMI' in X.columns:
        severity_scores += (X['BMI'] > 35).astype(int)
    
    # Rule 3: Poor physical health indicates complications
    if 'PhysHlth' in X.columns:
        severity_scores += (X['PhysHlth'] > 14).astype(int)  # More than 2 weeks of poor health
    
    # Rule 4: Combination of high blood pressure and high cholesterol
    if 'HighBP' in X.columns and 'HighChol' in X.columns:
        severity_scores += ((X['HighBP'] == 1) & (X['HighChol'] == 1)).astype(int)
    
    # Rule 5: Difficulty walking/climbing stairs indicates complications
    if 'DiffWalk' in X.columns:
        severity_scores += (X['DiffWalk'] == 1).astype(int)
    
    # Rule 6: Age-related severity (older age → higher risk of complications)
    if 'Age' in X.columns:
        severity_scores += (X['Age'] >= 10).astype(int)  # Age ≥ 65 (Age column is categorical in BRFSS)
    
    # Rule 7: Poor general health 
    if 'GenHlth' in X.columns:
        severity_scores += (X['GenHlth'] >= 4).astype(int)  # Fair or poor health
    
    # Classify based on severity score
    # Tunable threshold - this value determines the split between class 1 and 2
    severity_threshold = 2
    
    # If score meets or exceeds threshold, classify as severe (class 2)
    # Otherwise, classify as moderate (class 1)
    severity_class = np.where(severity_scores >= severity_threshold, 2, 1)
    
    return severity_class

# Define function to try different severity thresholds
def evaluate_severity_thresholds(X, y, thresholds=[1, 2, 3, 4]):
    """Evaluate different severity score thresholds for class 1 vs 2 determination"""
    results = []
    
    # Extract only actual positive cases (class 1 or 2)
    mask = y > 0
    X_positives = X[mask]
    y_positives = y[mask]
    
    for threshold in thresholds:
        # Set the severity threshold for the rules
        severity_scores = np.zeros(len(X_positives))
        
        # Calculate severity scores (same logic as determine_diabetes_severity)
        if 'HeartDiseaseorAttack' in X_positives.columns:
            severity_scores += (X_positives['HeartDiseaseorAttack'] == 1).astype(int) * 2
        
        if 'Stroke' in X_positives.columns:
            severity_scores += (X_positives['Stroke'] == 1).astype(int) * 2
        
        if 'BMI' in X_positives.columns:
            severity_scores += (X_positives['BMI'] > 35).astype(int)
        
        if 'PhysHlth' in X_positives.columns:
            severity_scores += (X_positives['PhysHlth'] > 14).astype(int)
        
        if 'HighBP' in X_positives.columns and 'HighChol' in X_positives.columns:
            severity_scores += ((X_positives['HighBP'] == 1) & (X_positives['HighChol'] == 1)).astype(int)
        
        if 'DiffWalk' in X_positives.columns:
            severity_scores += (X_positives['DiffWalk'] == 1).astype(int)
        
        if 'Age' in X_positives.columns:
            severity_scores += (X_positives['Age'] >= 10).astype(int)  # Age ≥ 65
        
        if 'GenHlth' in X_positives.columns:
            severity_scores += (X_positives['GenHlth'] >= 4).astype(int)
        
        # Apply threshold
        y_pred = np.where(severity_scores >= threshold, 2, 1)
        
        # Calculate recalls for class 1 and 2
        class1_recall = recall_score(y_positives == 1, y_pred == 1)
        class2_recall = recall_score(y_positives == 2, y_pred == 2)
        
        # Store results
        results.append({
            'threshold': threshold,
            'class1_recall': class1_recall,
            'class2_recall': class2_recall,
            'avg_recall': (class1_recall + class2_recall) / 2,
            'score_distribution': np.bincount(severity_scores.astype(int))
        })
    
    return results

# Evaluate different severity thresholds
threshold_results = evaluate_severity_thresholds(X_test, y_test, thresholds=[1, 2, 3, 4, 5])

# Print threshold evaluation results
print("\n----- Severity Threshold Evaluation -----")
for result in threshold_results:
    print(f"Threshold {result['threshold']}:")
    print(f"  Class 1 recall: {result['class1_recall']:.4f}")
    print(f"  Class 2 recall: {result['class2_recall']:.4f}")
    print(f"  Average recall: {result['avg_recall']:.4f}")
    print(f"  Score distribution: {result['score_distribution']}")

# Find best threshold
best_threshold = max(threshold_results, key=lambda x: x['avg_recall'])['threshold']
print(f"\nBest severity threshold: {best_threshold}")

# Function to make hierarchical predictions
def hierarchical_predict(X, binary_model, binary_threshold=0.2, severity_threshold=1):
    # Stage 1: Binary classification
    binary_probs = binary_model.predict_proba(X)[:, 1]
    binary_preds = (binary_probs >= binary_threshold).astype(int)
    
    # Initialize with all zeros
    final_preds = np.zeros(len(X))
    
    # Find samples predicted as positive (1 or 2)
    positive_indices = np.where(binary_preds == 1)[0]
    
    if len(positive_indices) > 0:
        # Stage 2: For positive predictions, determine severity
        if hasattr(X, 'iloc'):
            X_positives = X.iloc[positive_indices]
        else:
            X_positives = X[positive_indices]
        
        # Apply deterministic rules
        severity_scores = np.zeros(len(X_positives))
        
        # Calculate severity scores
        if 'HeartDiseaseorAttack' in X_positives.columns:
            severity_scores += (X_positives['HeartDiseaseorAttack'] == 1).astype(int) * 2
        
        if 'Stroke' in X_positives.columns:
            severity_scores += (X_positives['Stroke'] == 1).astype(int) * 2
        
        if 'BMI' in X_positives.columns:
            severity_scores += (X_positives['BMI'] > 35).astype(int)
        
        if 'PhysHlth' in X_positives.columns:
            severity_scores += (X_positives['PhysHlth'] > 14).astype(int)
        
        if 'HighBP' in X_positives.columns and 'HighChol' in X_positives.columns:
            severity_scores += ((X_positives['HighBP'] == 1) & (X_positives['HighChol'] == 1)).astype(int)
        
        if 'DiffWalk' in X_positives.columns:
            severity_scores += (X_positives['DiffWalk'] == 1).astype(int)
        
        if 'Age' in X_positives.columns:
            severity_scores += (X_positives['Age'] >= 10).astype(int)
        
        if 'GenHlth' in X_positives.columns:
            severity_scores += (X_positives['GenHlth'] >= 4).astype(int)
        
        # Apply threshold
        severity_preds = np.where(severity_scores >= severity_threshold, 2, 1)
        
        # Assign to final predictions
        final_preds[positive_indices] = severity_preds
    
    return final_preds

# Make hierarchical predictions with best threshold
y_pred_hierarchical = hierarchical_predict(
    X_test, binary_model, 
    binary_threshold=binary_threshold, 
    severity_threshold=best_threshold
)

# Evaluate final hierarchical model
print("\n----- Combined Hierarchical Model Evaluation -----")
print(classification_report(y_test, y_pred_hierarchical))

# Class-specific recall analysis
print("\nClass-specific Recall for Hierarchical Model:")
for cls in range(3):
    if cls in np.unique(y_test):
        recall = recall_score(y_test == cls, y_pred_hierarchical == cls)
        print(f"Class {cls} Recall: {recall:.4f}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred_hierarchical)
print("\nConfusion Matrix:")
print(cm)

# Analyze severity score distribution for class 1 and 2
mask_test_positives = y_test > 0
X_test_positives = X_test[mask_test_positives]
y_test_positives = y_test[mask_test_positives]

# Calculate severity scores for all positive cases
severity_scores = np.zeros(len(X_test_positives))

if 'HeartDiseaseorAttack' in X_test_positives.columns:
    severity_scores += (X_test_positives['HeartDiseaseorAttack'] == 1).astype(int) * 2

if 'Stroke' in X_test_positives.columns:
    severity_scores += (X_test_positives['Stroke'] == 1).astype(int) * 2

if 'BMI' in X_test_positives.columns:
    severity_scores += (X_test_positives['BMI'] > 35).astype(int)

if 'PhysHlth' in X_test_positives.columns:
    severity_scores += (X_test_positives['PhysHlth'] > 14).astype(int)

if 'HighBP' in X_test_positives.columns and 'HighChol' in X_test_positives.columns:
    severity_scores += ((X_test_positives['HighBP'] == 1) & (X_test_positives['HighChol'] == 1)).astype(int)

if 'DiffWalk' in X_test_positives.columns:
    severity_scores += (X_test_positives['DiffWalk'] == 1).astype(int)

if 'Age' in X_test_positives.columns:
    severity_scores += (X_test_positives['Age'] >= 10).astype(int)

if 'GenHlth' in X_test_positives.columns:
    severity_scores += (X_test_positives['GenHlth'] >= 4).astype(int)

# Group by class and get score distribution
class1_scores = severity_scores[y_test_positives == 1]
class2_scores = severity_scores[y_test_positives == 2]

print("\nSeverity Score Distribution for Class 1:")
print(np.bincount(class1_scores.astype(int)) / len(class1_scores))

print("\nSeverity Score Distribution for Class 2:")
print(np.bincount(class2_scores.astype(int)) / len(class2_scores))

# Save the binary model
binary_model.save_model('catboost_binary_diabetes.cbm')

# Create a standalone function for diabetes severity prediction
def predict_diabetes_severity(X_new, binary_model_path='catboost_binary_diabetes.cbm', 
                             binary_threshold=0.2, severity_threshold=best_threshold):
    """
    Predict diabetes status and severity using the hybrid model.
    
    Parameters:
    -----------
    X_new : DataFrame
        New features to predict on
    binary_model_path : str
        Path to saved binary model
    binary_threshold : float
        Threshold for binary model
    severity_threshold : int
        Threshold for severity rules
    
    Returns:
    --------
    numpy.ndarray
        Predictions: 0 = no diabetes, 1 = moderate diabetes, 2 = severe diabetes
    """
    # Load binary model
    binary_model = CatBoostClassifier()
    binary_model.load_model(binary_model_path)
    
    # Make predictions using the hierarchical model
    return hierarchical_predict(X_new, binary_model, binary_threshold, severity_threshold)

print("\nExample usage of prediction function:")
print("predictions = predict_diabetes_severity(new_data)")

Available features: ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income', 'BMI_X_Age', 'HighBP_X_HighChol', 'GenHlth_X_PhysHlth', 'BMI_X_PhysActivity', 'Age_X_HighBP', 'Income_X_Education', 'Age_X_DiffWalk', 'health_bp_decay', 'low_ses', 'condition_count', 'health_cluster', 'risk_score_mult']
Class distribution in training set: [170962   3705  28277]
Binary class distribution in training set: [170962  31982]
0:	learn: 0.6715087	test: 0.6717505	best: 0.6717505 (0)	total: 48.1ms	remaining: 48.1s
100:	learn: 0.4861871	test: 0.4910353	best: 0.4910353 (100)	total: 4.97s	remaining: 44.2s
200:	learn: 0.4823351	test: 0.4892199	best: 0.4892199 (200)	total: 9.73s	remaining: 38.7s
300:	learn: 0.4789850	test: 0.4885578	best: 0.4885550 (294)	total: 14.1s	remaining: 32.8s
400:	learn: 0.4758785	te

In [60]:

from collections import Counter

# Load data
data = pd.read_csv(r'D:\PIMA\data_2\cleaned_3.csv')  # Adjust path as needed

# Set up features and target
X = data.drop('Diabetes_012', axis=1)  # Adjust column name as needed
y = data['Diabetes_012'].astype(int)  # Ensure this has 0, 1, 2 values

# Reset indices to ensure they are continuous integers starting from 0
X = X.reset_index(drop=True)
y = pd.Series(y.values).reset_index(drop=True)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Reset indices again after splitting to avoid any indexing issues
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = pd.Series(y_train.values).reset_index(drop=True)
y_test = pd.Series(y_test.values).reset_index(drop=True)

# Print class distribution
train_counts = np.bincount(y_train.astype(int))
print(f"Training set class distribution:")
for cls, count in enumerate(train_counts):
    print(f"Class {cls}: {count} samples ({count/len(y_train)*100:.2f}%)")

# Function to create balanced subsamples
def create_balanced_subset(X, y, n_estimators=10, sampling_ratio=1.5):
    """
    Create multiple balanced datasets by randomly subsampling class 0
    
    Parameters:
    -----------
    X : DataFrame
        Features
    y : Series or array
        Target with class 0, 1, 2
    n_estimators : int
        Number of subsampled datasets to create
    sampling_ratio : float
        Ratio of class 0 samples to class 1+2 samples
        
    Returns:
    --------
    List of tuples (X_subset, y_subset) for each balanced dataset
    """
    # Get indices for each class
    idx_class0 = np.where(y == 0)[0]
    idx_class1 = np.where(y == 1)[0]
    idx_class2 = np.where(y == 2)[0]
    
    # Count samples in each class
    n_class0 = len(idx_class0)
    n_class1 = len(idx_class1)
    n_class2 = len(idx_class2)
    n_minority = n_class1 + n_class2
    
    # Calculate how many class 0 samples to include in each subset
    n_samples_class0 = int(n_minority * sampling_ratio)
    
    # Make sure we don't try to sample more than available
    n_samples_class0 = min(n_samples_class0, n_class0)
    
    # Create multiple balanced subsets
    subsets = []
    for i in range(n_estimators):
        # Randomly sample class 0 indices without replacement
        sampled_idx_class0 = np.random.choice(idx_class0, size=n_samples_class0, replace=False)
        
        # Combine with all samples from class 1 and 2
        subset_indices = np.concatenate([sampled_idx_class0, idx_class1, idx_class2])
        
        # Create balanced subset - use iloc to index by position, not by label
        X_subset = X.iloc[subset_indices]
        y_subset = y.iloc[subset_indices]
        
        # Check class distribution in subset
        subset_counts = np.bincount(y_subset.astype(int))
        print(f"Subset {i+1} class distribution: {subset_counts}")
        
        subsets.append((X_subset, y_subset))
    
    return subsets

# Create balanced subsets for training
n_estimators = 5  # Number of models in the ensemble
subsets = create_balanced_subset(X_train, y_train, n_estimators=n_estimators, sampling_ratio=2.0)

# Train a model on each balanced subset
models = []
for i, (X_subset, y_subset) in enumerate(subsets):
    print(f"\nTraining model {i+1}/{n_estimators}...")
    
    # Initialize CatBoost classifier
    model = CatBoostClassifier(
        iterations=500,  # Reduced iterations for faster training
        learning_rate=0.05,
        depth=6,
        loss_function='MultiClass',  # Multiclass classification
        random_seed=42 + i,  # Different seed for each model
        verbose=100
    )
    
    # Train model
    model.fit(X_subset, y_subset)
    
    # Add to ensemble
    models.append(model)
    
    # Evaluate individual model
    y_pred = model.predict(X_test)
    print(f"\nModel {i+1} performance:")
    print(classification_report(y_test, y_pred))

# Function for ensemble prediction - hard voting
def ensemble_predict_hard_voting(models, X):
    """Predict using majority voting from all models"""
    # Get predictions from each model
    predictions = np.array([model.predict(X) for model in models])
    
    # Transpose to get predictions by sample
    predictions = predictions.T
    
    # For each sample, count votes for each class
    ensemble_predictions = np.zeros(len(X))
    
    for i, preds in enumerate(predictions):
        # Count occurrences of each class
        votes = Counter(preds)
        
        # Find class with most votes (majority)
        ensemble_predictions[i] = votes.most_common(1)[0][0]
    
    return ensemble_predictions

# Function for ensemble prediction - soft voting (averaging probabilities)
def ensemble_predict_soft_voting(models, X, weights=None):
    """Predict using weighted average of probabilities from all models"""
    if weights is None:
        weights = np.ones(len(models)) / len(models)
    
    # Get probability predictions from each model
    probas = np.array([model.predict_proba(X) for model in models])
    
    # Weight each model's predictions
    weighted_probas = np.array([w * p for w, p in zip(weights, probas)])
    
    # Sum weighted probabilities
    avg_probas = np.sum(weighted_probas, axis=0)
    
    # Return class with highest probability
    return np.argmax(avg_probas, axis=1)

# Function for weighted ensemble prediction with class-specific focus
def ensemble_predict_class_weighted(models, X, class1_weight=2.0, class2_weight=1.5):
    """
    Predict using weighted probabilities that prioritize class 1 and 2 detection
    
    Parameters:
    -----------
    models : list
        List of trained models
    X : DataFrame
        Features to predict on
    class1_weight : float
        Weight multiplier for class 1 probabilities
    class2_weight : float
        Weight multiplier for class 2 probabilities
        
    Returns:
    --------
    numpy.ndarray
        Predictions: 0, 1, or 2
    """
    # Get probability predictions from each model
    all_probas = np.array([model.predict_proba(X) for model in models])
    
    # Average probabilities across models
    avg_probas = np.mean(all_probas, axis=0)
    
    # Apply class-specific weights to prioritize class 1 and 2
    weighted_probas = avg_probas.copy()
    weighted_probas[:, 1] *= class1_weight  # Boost class 1 probabilities
    weighted_probas[:, 2] *= class2_weight  # Boost class 2 probabilities
    
    # Return class with highest weighted probability
    return np.argmax(weighted_probas, axis=1)

# Evaluate ensemble with hard voting (majority vote)
y_pred_hard = ensemble_predict_hard_voting(models, X_test)
print("\n----- Ensemble with Hard Voting -----")
print(classification_report(y_test, y_pred_hard))

# Evaluate ensemble with soft voting (probability averaging)
y_pred_soft = ensemble_predict_soft_voting(models, X_test)
print("\n----- Ensemble with Soft Voting -----")
print(classification_report(y_test, y_pred_soft))

# Evaluate ensemble with class-weighted voting
# Try different weights to prioritize class 1 and 2
weight_combinations = [
    (1.5, 1.2),  # Moderate boost for classes 1 and 2
    (2.0, 1.5),  # Higher boost for class 1 than class 2
    (3.0, 2.0),  # Stronger boost for both
    (4.0, 2.0),  # Very strong boost for class 1
    (2.0, 4.0)   # Very strong boost for class 2
]

print("\n----- Testing Different Class Weight Combinations -----")
best_avg_recall = 0
best_weights = None
best_predictions = None

for class1_w, class2_w in weight_combinations:
    y_pred_weighted = ensemble_predict_class_weighted(models, X_test, class1_weight=class1_w, class2_weight=class2_w)
    
    # Calculate class-specific recalls
    recall_0 = recall_score(y_test == 0, y_pred_weighted == 0)
    recall_1 = recall_score(y_test == 1, y_pred_weighted == 1)
    recall_2 = recall_score(y_test == 2, y_pred_weighted == 2)
    
    # Calculate average recall for classes 1 and 2
    avg_recall_1_2 = (recall_1 + recall_2) / 2
    
    print(f"\nClass weights: Class 1 = {class1_w}, Class 2 = {class2_w}")
    print(f"Class 0 recall: {recall_0:.4f}")
    print(f"Class 1 recall: {recall_1:.4f}")
    print(f"Class 2 recall: {recall_2:.4f}")
    print(f"Average recall (classes 1 & 2): {avg_recall_1_2:.4f}")
    
    # Check if this is the best combination so far
    if avg_recall_1_2 > best_avg_recall:
        best_avg_recall = avg_recall_1_2
        best_weights = (class1_w, class2_w)
        best_predictions = y_pred_weighted

# Use best weights for final evaluation
print(f"\n----- Best Weight Combination: Class 1 = {best_weights[0]}, Class 2 = {best_weights[1]} -----")
print(classification_report(y_test, best_predictions))

# Class-specific recall analysis for best model
print("\nClass-specific Recall for Best Ensemble Model:")
for cls in range(3):
    if cls in np.unique(y_test):
        recall = recall_score(y_test == cls, best_predictions == cls)
        print(f"Class {cls} Recall: {recall:.4f}")

# Custom performance metric that prioritizes class 1 and 2 recall
def calculate_custom_score(y_true, y_pred):
    recall_0 = recall_score(y_true == 0, y_pred == 0)
    recall_1 = recall_score(y_true == 1, y_pred == 1)
    recall_2 = recall_score(y_true == 2, y_pred == 2)
    
    # Custom score that prioritizes class 2, then class 1, and penalizes high class 0 recall
    score = (recall_2 * 0.6) + (recall_1 * 0.4) - (recall_0 * 0.2)
    
    return score, recall_0, recall_1, recall_2

# Calculate custom score for best ensemble
custom_score, r0, r1, r2 = calculate_custom_score(y_test, best_predictions)
print(f"\nCustom Score (prioritizing classes 1 & 2): {custom_score:.4f}")
print(f"Class 0 recall: {r0:.4f}")
print(f"Class 1 recall: {r1:.4f}")
print(f"Class 2 recall: {r2:.4f}")

# Save the ensemble models
for i, model in enumerate(models):
    model.save_model(f'catboost_ensemble_model_{i+1}.cbm')

# Function to make predictions using the ensemble
def predict_with_ensemble(X_new, model_paths=None, class1_weight=best_weights[0], class2_weight=best_weights[1]):
    """
    Predict using the ensemble of models with class-weighted voting
    
    Parameters:
    -----------
    X_new : DataFrame
        New features to predict on
    model_paths : list
        List of paths to saved models (if None, uses default paths)
    class1_weight : float
        Weight for class 1
    class2_weight : float
        Weight for class 2
    
    Returns:
    --------
    numpy.ndarray
        Predictions: 0, 1, or 2
    """
    if model_paths is None:
        model_paths = [f'catboost_ensemble_model_{i+1}.cbm' for i in range(n_estimators)]
    
    # Load models
    loaded_models = []
    for path in model_paths:
        model = CatBoostClassifier()
        model.load_model(path)
        loaded_models.append(model)
    
    # Make class-weighted predictions
    return ensemble_predict_class_weighted(loaded_models, X_new, 
                                          class1_weight=class1_weight, 
                                          class2_weight=class2_weight)

print("\nExample usage of ensemble prediction function:")
print("predictions = predict_with_ensemble(new_data)")

Training set class distribution:
Class 0: 170962 samples (84.24%)
Class 1: 3705 samples (1.83%)
Class 2: 28277 samples (13.93%)
Subset 1 class distribution: [63964  3705 28277]
Subset 2 class distribution: [63964  3705 28277]
Subset 3 class distribution: [63964  3705 28277]
Subset 4 class distribution: [63964  3705 28277]
Subset 5 class distribution: [63964  3705 28277]

Training model 1/5...
0:	learn: 1.0547578	total: 116ms	remaining: 58s
100:	learn: 0.5921423	total: 5.3s	remaining: 21s
200:	learn: 0.5853564	total: 9.55s	remaining: 14.2s
300:	learn: 0.5801343	total: 14s	remaining: 9.23s
400:	learn: 0.5757723	total: 18.1s	remaining: 4.47s
499:	learn: 0.5718670	total: 22.3s	remaining: 0us

Model 1 performance:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     42741
           1       0.00      0.00      0.00       926
           2       0.40      0.55      0.46      7069

    accuracy                           0.81     50736
   macro 

d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


100:	learn: 0.5928815	total: 4.85s	remaining: 19.2s
200:	learn: 0.5858318	total: 8.95s	remaining: 13.3s
300:	learn: 0.5807847	total: 13.3s	remaining: 8.78s
400:	learn: 0.5762521	total: 17.5s	remaining: 4.32s
499:	learn: 0.5724593	total: 21.8s	remaining: 0us

Model 2 performance:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     42741
           1       0.00      0.00      0.00       926
           2       0.40      0.55      0.46      7069

    accuracy                           0.81     50736
   macro avg       0.44      0.47      0.45     50736
weighted avg       0.82      0.81      0.81     50736


Training model 3/5...


d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0:	learn: 1.0547412	total: 60.1ms	remaining: 30s
100:	learn: 0.5916045	total: 5.01s	remaining: 19.8s
200:	learn: 0.5842809	total: 9.41s	remaining: 14s
300:	learn: 0.5790827	total: 14.1s	remaining: 9.3s
400:	learn: 0.5744641	total: 18.6s	remaining: 4.58s
499:	learn: 0.5705095	total: 24.5s	remaining: 0us

Model 3 performance:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     42741
           1       0.00      0.00      0.00       926
           2       0.40      0.56      0.46      7069

    accuracy                           0.81     50736
   macro avg       0.44      0.48      0.45     50736
weighted avg       0.82      0.81      0.81     50736


Training model 4/5...


...

0:	learn: 1.0541207	total: 69.3ms	remaining: 34.6s
100:	learn: 0.5920090	total: 7.96s	remaining: 31.4s
200:	learn: 0.5849939	total: 16.3s	remaining: 24.2s
300:	learn: 0.5799839	total: 23.8s	remaining: 15.7s
400:	learn: 0.5756543	total: 28.4s	remaining: 7.01s
499:	learn: 0.5716707	total: 32.8s	remaining: 0us

Model 4 performance:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     42741
           1       0.00      0.00      0.00       926
           2       0.40      0.56      0.46      7069

    accuracy                           0.81     50736
   macro avg       0.44      0.48      0.45     50736
weighted avg       0.82      0.81      0.81     50736


Training model 5/5...


d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0:	learn: 1.0553114	total: 41.1ms	remaining: 20.5s
100:	learn: 0.5941558	total: 4.94s	remaining: 19.5s
200:	learn: 0.5872655	total: 9.41s	remaining: 14s
300:	learn: 0.5821000	total: 13.6s	remaining: 8.98s
400:	learn: 0.5779081	total: 17.6s	remaining: 4.34s
499:	learn: 0.5739701	total: 21.9s	remaining: 0us

Model 5 performance:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     42741
           1       0.00      0.00      0.00       926
           2       0.40      0.55      0.46      7069

    accuracy                           0.81     50736
   macro avg       0.44      0.47      0.45     50736
weighted avg       0.82      0.81      0.81     50736



d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\PIMA\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TypeError: unhashable type: 'numpy.ndarray'